In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.22.0"
#r "nuget: Microsoft.SemanticKernel.Prompty, 1.22.0-alpha"

Installed Packages Microsoft.SemanticKernel, 1.22.0 Microsoft.SemanticKernel.Prompty, 1.22.0-alpha

In [2]:
#r "Interactive.Prompty/Interactive.Prompty/bin/Debug/net8.0/Interactive.Prompty.dll"

In [3]:
using System.Diagnostics;
Debugger.Launch();

In [4]:
using Microsoft.DotNet.Interactive;
using Interactive.Prompty;


await PromptyExtension.LoadExtensionAsync((CompositeKernel)Kernel.Root);

In [5]:
#!connect prompty --kernel-name prompty

Kernel added: #!prompty

In [6]:
#!connect promptyOrchestrator --kernel-name orchestrator-kernel --prompty-kernel-name prompty --azure-openai-endpoint @input:{"saveAs":"azure-openai-endpoint"} --azure-openai-deployment-name @input:{"saveAs":"azure-openai-deployment-name"} --azure-openai-apikey @password:{"saveAs":"azure-openai-apikey"}

Kernel added: #!orchestrator-kernel

In [7]:
---
name: ExamplePrompt
description: A prompt that uses context to ground an incoming question
authors:
  - Seth Juarez
model:
  api: chat
  parameters:
    max_tokens: 3000
sample:
  firstName: Gino
  context: >
    The Alpine Explorer Tent boasts a detachable divider for privacy,
    numerous mesh windows and adjustable vents for ventilation, and
    a waterproof design. It even has a built-in gear loft for storing
    your outdoor essentials. In short, it's a blend of privacy, comfort,
    and convenience, making it your second home in the heart of nature!
  question: What can you tell me about your tents?
---
 
system:
You are an AI assistant who helps people find information. As the assistant,
you answer questions briefly, succinctly, and in a personable manner using
markdown and even add some personal flair with appropriate emojis.
 
# Customer
You are helping {{firstName}} to find answers to their questions.
Use their name to address them in your responses.
 
# Context
Use the following context to provide a more personalized response to {{firstName}}:
{{context}}
 
user:
{{input}}
 
 

In [9]:
what is new? but about the people in the nite

Hey there! 🌙 It sounds like you're curious about the latest happenings or trends within nightlife. Here’s a quick rundown:

1. **Live Music**: Many venues are focusing on local artists and unique genres to create an intimate experience.
2. **Themed Events**: Theme nights are all the rage, adding a fun twist to your typical outings!
3. **Interactive Experiences**: Bars and clubs are introducing activities like karaoke, trivia nights, and immersive art displays.

Is there anything specific about nightlife or a certain location you're interested in? Let me know! 🎉